## pyKO 

Example script to generate silica vapor plume grid results

In [1]:
%run ../../import-modules-grid

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

!jupyter --version

Platform:  Darwin Kernel Version 24.0.0: Tue Sep 24 23:37:25 PDT 2024; root:xnu-11215.1.12~1/RELEASE_ARM64_T6030
python version:  3.11.9
matplotlib version:  3.9.1
hvplot version:  0.10.0
numpy version:  2.0.1
pandas version:  2.2.2
pickle version:  4.0
yaml version:  6.0.2
pint version:  0.24.3
pyko version:  v0.8.3-dev-2024-05-12
print eos_table version:  v1.1.5b

Number of CPUs in the system: 12


Selected Jupyter core packages...
IPython          : 8.26.0
ipykernel        : 6.29.5
ipywidgets       : 8.1.3
jupyter_client   : 8.6.2
jupyter_core     : 5.7.2
jupyter_server   : 2.14.2
jupyterlab       : 4.2.4
nbclient         : 0.10.0
nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : not installed
qtconsole        : not installed
traitlets        : 5.14.3


## Example of checking EOS table initial state and verifying input file OK

EOS Tables are initialized based on the user input density and internal energy (you could change that if desired). In v0.4, the input P and T are over-ridden by the EOS Table state variables for the given density and energy.

In [3]:
# start with your initial guess of the EOS initial state 
# load and print the configuration file without running the simulation
# 
filein = './vp-silica-grid-1Pa/vp-silica-sphere-grid.yml'
plotbase = './vp-silica-grid-1Pa/'
# initialize the run class variable by loading the configuration file
# this loads the requires EOS tables, so you can inquire about initial states
run = RunClass(fin=filein)
# print the run class state; this will print in code units
run.checkinput()


Fracture is turned off for mat1
Fracture is turned off for mat2
found pvoid

pyKO v0.8.3-dev-2024-05-12 run parameters
   All outputs are in code units 
   Input file: ./vp-silica-grid-1Pa/vp-silica-sphere-grid.yml 
   Output file: ./vp-silica-grid-1Pa/grid- 
   Number of materials: 2 
   Number of nodes in each material: [ 50 200] 
   Length of each material: [2.5e+06 5.0e+10] 
   Initial left edge of each material: [      0. 2500000.] 
   Boundary conditions: ['FIXED', 'FIXED']
   Material EOS:     ['SES', 'IDG'] 
   Geometry:         SPH 
   Gravity:          0.0 
   Void pressure:    1.0000000000000002e-12 
   Time step factor: 6 
   Stop time:        86400000000.0

mat1: Hydrodynamic material

mat1 Fracture parameters [code units]: 
   Fracture is turned on:                       False 
   Fracture pressure:                           1e+99 
   Fracture maximum distension (rhomin/rhoref): 0.8

Class SESAME [code units]: ANEOS Fused Silica v0.2 G11 
   eos_table module version v1.1.

In [3]:
%%capture cap

rplumeinitarr = np.asarray([1.e3,5.e3,25.e3,125.e3]) # m
rnebinitarr = np.asarray([10.e6,50.e6,250.e6,1250.e6]) #m
pinitarr = np.asarray([100.e9,150.e9,200.e9,250.e9, 350.e9, 450.e9]) #Pa 
velinitarr = np.asarray([0.])#,2.e3,4.e3]) # m/s
tsavearr = np.asarray([5.,25.,125.,600.])/2 #s

tstall = np.zeros((len(rplumeinitarr),len(pinitarr),len(velinitarr)))

if 1:
    fin='./vp-silica-grid-1Pa/vp-silica-sphere-grid.yml'
    fout='./vp-silica-grid-1Pa/vp-silica-sphere-grid-'
    tend = 86400.e6*15 # initial stop time
    ftype='YAML'
    verbose=True
    userdtstart=0.
    usertstepscale=0.
    binoutput=True
    debug=False
    initarr=False
    run = RunClass(fin=fin,fout=fout,ftype=ftype)    # initialize run parameters class
    #
    # read in the run template
    readinput_yaml(run,verbose=verbose)
    print(run.ieos[0].path)
    hugarr = np.loadtxt(run.ieos[0].path+'NEW-SESAME-HUG-E.TXT',skiprows=3,delimiter=',')
    print('Hugoniot file size = ',hugarr.shape)
    # vary starting parameters
    for ipp in range(len(pinitarr)):
        # figure out the initial state on the principal Hugoniot
        rhostart = np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,1])
        iestart =  np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,3])
        tempstart =  np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,0])
        entstart =  np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,4])
        upplanarstart =  np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,5])
        print('initial state iii, P, rho, ie, T [GPa, g/cm3, MJ/kg] = ',ipp,pinitarr[ipp]/1.e9,rhostart,iestart,tempstart)
        print('initial state iii, S [J/K/kg] = ',entstart*1.e3)
        print('equivalent planar up, planar vi (km/s) = ',upplanarstart,upplanarstart*2)
        ureg.define('eu = 1.0E12 ergs') 
        utmp = Q_(iestart,'MJ/kg') # creates a variable utmp that has pint units information
        print('internal energy in code units eu/g: ',utmp.to('eu/g')) # prints the value and units
        print('internal energy in mks units: ',utmp.to('J/kg')) # prints the value and units
        ptmp = Q_(pinitarr[ipp]/1.e9,'GPa') # creates a variable ptmp that has pint units information
        print('pressure in code units Mbar: ',ptmp.to('Mbar')) # prints the value and units
        print('pressure in mks (Pa) units: ',ptmp.to('Pa')) # prints the value and units
        print('pressure in mks (GPa) units: ',ptmp.to('GPa')) # prints the value and units
        #initarr = [upstart,rhostart,pstart,tstart,ievstart,csstart] # code units
        rtmp = Q_(rhostart,'g/cm^3')
        for irp in range(len(rplumeinitarr)):
            for ivel in range(len(velinitarr)):            
                run = RunClass(fin=fin,fout=fout,ftype=ftype)    # initialize run parameters class
                readinput_yaml(run)#verbose=verbose)
                fileid = 'P'+str(np.round(pinitarr[ipp]/1.e9))+'-R'+str(np.round(rplumeinitarr[irp]/1.e3))+'-V'+str(np.round(velinitarr[ivel]/1.e3))
                run.outputfilename = fout+fileid+'.dat'
                print('outputfile',run.outputfilename)
                initarr = [velinitarr[ivel]/10.e3,rtmp.magnitude,ptmp.to('Mbar').magnitude,utmp.to('eu/g').magnitude*rtmp.magnitude,tempstart]
                print(initarr)
                run.ixstart[1]=rplumeinitarr[irp]*1.e2 # cm
                run.ilength[0]=rplumeinitarr[irp]*1.e2 # cm
                run.ilength[1]=rnebinitarr[irp]*1.e2 # cm
                print('length vars = ',run.ixstart,run.ilength,run.inodes)
                run.time_skip = tsavearr[irp]*1.e6 # microsec
                print('Save output every microsec =',run.time_skip)
                
                ##############################################################
                if 1:
                    print('\n pyKO STARTING RUN \n')
                    run.tstop=tend
                    # initialize problem domain data structure
                    # pass in the problem run parameters
                    data = DomainClass() # create main data structure
                    # set the initial velocity of the vapor plume
                    # using code units for now; velocity units are 10 km/s
                    if initarr != False:
                        #initarr = [upstart,rhostart,pstart,ievstart,tempstart] # code units
                        run.iupstart[0]=initarr[0]
                        run.irhostart[0]=initarr[1]
                        run.ipstart[0]=initarr[2]
                        run.iiev0start[0]=initarr[3]            
                        run.itempstart[0]=initarr[4]
                        # not filling in sound speed
                        run.ieos[0].p0     = initarr[2]
                        run.ieos[0].up0    = initarr[0]
                        run.ieos[0].rho0   = initarr[1]
                        run.ieos[0].iev0   = initarr[4]
                        run.ieos[0].EOS.R0REF = initarr[1]
                        #run.outputfilename = run.outputfilename+'-'+str(initarr[0])+'-'+str(round(initarr[2]*100))
                    print('initial up = ',run.iupstart)
                    print('initial p = ',run.ipstart)
                    print('initial rho = ',run.irhostart)
                    print('initial ie = ',run.iiev0start)
                    print('outputfile',run.outputfilename)
                    data.makegrid(run,verbose=verbose) # create initial grid; also calculates a guess for the initial time step
                    if userdtstart > 0.:
                        # for testing comparisons to fKO with fixed initial time step
                        if verbose: print('Using user supplied initial time step (microsec): ',userdtstart)
                        run.dtstart = userdtstart
                        data.deltat   = run.dtstart      # current time step; while in time loop
                        data.deltat_next = run.dtstart   # next time step; calculated at end of time loop
                    if usertstepscale > 0.:
                        if verbose: print('Using user supplied reduction factor for time steps: ',usertstepscale)
                        run.tstepscale = usertstepscale
                    run.debugflag = debug
                    data.writefirstoutput(run,checkexistsflag=False,verbose=verbose,overwriteflag=True) # write initial grid to outputfile - right now just the header
                    
                    # testing: can call one time step only or a fixed number of time steps to check the simulation parameters
                    # advance one time step
                    #data.advancetime(run)
                    #for steps in range(20):
                    #    data.advancetime(run,verbose=verbose)
                        
                    # main time loop
                    print('pyKO is running.....')
                    t0 = pytime.time()
                    # run once
                    if 0:
                        data.advancetime(run,verbose=verbose)
                    if 1:
                        runflag=True
                        while data.time[3] < run.tstop:
                        #while runflag:
                            data.advancetime(run,verbose=verbose)
                            mat1negup = np.where((data.up[2,:] < -1.e-4))[0] # < 1 m/s
                            #imat1max = np.max(np.where(data.matid ==0)[0])
                            #if runflag and (data.up[2,imat1max] < 0.):
                            if runflag and (len(mat1negup)>0):
                                runflag=False
                                print(mat1negup)
                                # microsec to sec; cm to Mm
                                print('STALL TIME (s), position (Mm) = ',data.time[3]/1.e6,data.pos[2,mat1negup[0]]/1.e2/1.e6)
                                data.onebinaryoutputpint(run,filetag='-stall')
                                run.tstop = data.time[3]*5
                    #
                    if data.stepn > max(run.outputsteps): # write last time step if needed
                        data.binaryoutputpint(run) # pickle output 
                    t1 = pytime.time()
                    
                    # diagnostic energy checks and run time information
                    print('Zeroth and final mvtotal: ',run.outputmvtot[0],run.outputmvtot[-1])
                    print('Zeroth and final KE+IE: ', \
                          run.outputietot[0]+run.outputketot[0],run.outputietot[-1]+run.outputketot[-1])
                    print('KE+IE ratio (final/zeroth)):', \
                          (run.outputietot[-1]+run.outputketot[-1])/(run.outputietot[0]+run.outputketot[0]))
                    print('First output step and final output step KE+IE: ', \
                          run.outputietot[1]+run.outputketot[1],run.outputietot[-1]+run.outputketot[-1])
                    print('KE+IE ratio (final/first)):', \
                          (run.outputietot[-1]+run.outputketot[-1])/(run.outputietot[1]+run.outputketot[1]))
                    print('\n pyKO FINISHED RUN, simulation time = ',data.time[1+2],' us \n')
                    print(' pyKO Wall Clock Main Loop Run Time=', t1-t0, ' s')
                    print(run.outputfilename)


In [ ]:
from datetime import datetime

# get current datetime
today = datetime.now()
print('Today Datetime:', today)

# Get current ISO 8601 datetime in string format
iso_date = today.isoformat()
print('ISO DateTime:', iso_date)

Save the captured output to a text file
with open('vp-silica-sphere-grid-1Pa-output'+iso_date'+'.txt', 'w') as file:
    file.write(cap.stdout)